# Updating training data with new descriptions

Takes a (labelled or unlabelled) training data file in; maps descriptions to new descriptions from Elasticsearch; outputs training data file (.xlsx) with these new descriptions.

In [9]:
import sys
sys.path.append("../../..")

import pandas as pd
from heritageconnector import datastore
from tqdm.auto import tqdm

In [6]:
ES_INDEX = "heritageconnector_vanda_test"
TRAIN_DATA_IN_PATH = "../../../GITIGNORE_DATA/vanda_hc_data/NEL/nel_train_data_manual_20210829-0102_annotated.xlsx"

df = pd.read_excel(TRAIN_DATA_IN_PATH, index_col=0)
df.head()

,item_uri,item_uri_link,candidate_rank,item_description,ent_label,ent_text,candidate_title,candidate_type,candidate_uri,candidate_uri_link,link_correct,candidate_description
0,https://api.vam.ac.uk/v2/objects/search?id_per...,https://api.vam.ac.uk/v2/objects/search?id_per...,0,Ida and (Mary) Louise Cook were born in Sunder...,PERSON,Ida,IDA,ORGANISATION,https://api.vam.ac.uk/v2/objects/search?id_org...,https://api.vam.ac.uk/v2/objects/search?id_org...,0.0,Publisher
1,https://api.vam.ac.uk/v2/objects/search?id_per...,https://api.vam.ac.uk/v2/objects/search?id_per...,1,Ida and (Mary) Louise Cook were born in Sunder...,PERSON,Ida,Ida Dashwood Lady Ida Dashwood,PERSON,https://api.vam.ac.uk/v2/objects/search?id_per...,https://api.vam.ac.uk/v2/objects/search?id_per...,0.0,NaN
2,https://api.vam.ac.uk/v2/objects/search?id_per...,https://api.vam.ac.uk/v2/objects/search?id_per...,2,Ida and (Mary) Louise Cook were born in Sunder...,PERSON,Ida,Ida Carrara,PERSON,https://api.vam.ac.uk/v2/objects/search?id_per...,https://api.vam.ac.uk/v2/objects/search?id_per...,0.0,NaN
3,https://api.vam.ac.uk/v2/objects/search?id_per...,https://api.vam.ac.uk/v2/objects/search?id_per...,3,Ida and (Mary) Louise Cook were born in Sunder...,PERSON,Ida,Ida Carroll,PERSON,https://api.vam.ac.uk/v2/objects/search?id_per...,https://api.vam.ac.uk/v2/objects/search?id_per...,0.0,NaN
4,https://api.vam.ac.uk/v2/objects/search?id_per...,https://api.vam.ac.uk/v2/objects/search?id_per...,4,Ida and (Mary) Louise Cook were born in Sunder...,PERSON,Ida,Ida Perrin,PERSON,https://api.vam.ac.uk/v2/objects/search?id_per...,https://api.vam.ac.uk/v2/objects/search?id_per...,0.0,NaN


In [5]:
item_uris = df['item_uri'].unique().tolist()
candidate_uris = df['candidate_uri'].unique().tolist()

for the items we're running NER on, we use the *description*. for the link candidates, we use the *disambiguating description*

In [20]:
item_uri_description_mapping = dict()
candidate_uri_description_mapping = dict()

print('item uris')
for uri in tqdm(item_uris):
    item_uri_description_mapping[uri] = datastore.get_by_uri(ES_INDEX, uri)['_source']['data'].get('http://www.w3.org/2001/XMLSchema#description', "")
    
print('candidate uris')
for uri in tqdm(candidate_uris):
    candidate_uri_description_mapping[uri] = datastore.get_by_uri(ES_INDEX, uri)['_source']['data'].get('https://schema.org/disambiguatingDescription', "")

item uris


  0%|          | 0/9308 [00:00<?, ?it/s]

candidate uris


  0%|          | 0/52085 [00:00<?, ?it/s]

In [21]:
newdf = df.copy()

newdf['item_description'] = newdf['item_uri'].map(item_uri_description_mapping)
newdf['candidate_description'] = newdf['candidate_uri'].map(candidate_uri_description_mapping)

In [28]:
newdf.to_excel("../../../GITIGNORE_DATA/vanda_hc_data/NEL/nel_train_data_manual_20210829-0102_annotated_new_descriptions.xlsx")